<a href="https://colab.research.google.com/github/Vishy-A/DeepLearning/blob/main/HW4/4106HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import random
import torch.nn.functional as F
import torchtext

In [132]:
SOS_token = 0
EOS_token = 1

In [133]:
engtofra = [
    ("I am cold", "J'ai froid"),

    ("You are tired", "Tu es fatigué"),

    ("He is hungry", "Il a faim"),

    ("She is happy", "Elle est heureuse"),

    ("We are friends", "Nous sommes amis"),

    ("They are students", "Ils sont étudiants"),

    ("The cat is sleeping", "Le chat dort"),

    ("The sun is shining", "Le soleil brille"),

    ("We love music", "Nous aimons la musique"),

    ("She speaks French fluently", "Elle parle français couramment"),

    ("He enjoys reading books", "Il aime lire des livres"),

    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),

    ("The movie starts at 7 PM", "Le film commence à 19 heures"),

    ("She wears a red dress", "Elle porte une robe rouge"),

    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),

    ("He drives a blue car", "Il conduit une voiture bleue"),

    ("They visit museums often", "Ils visitent souvent des musées"),

    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),

    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),

    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),

    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),

    ("They travel around the world", "Ils voyagent autour du monde"),

    ("The book is on the table", "Le livre est sur la table"),

    ("She dances gracefully", "Elle danse avec grâce"),

    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),

    ("He works hard every day", "Il travaille dur tous les jours"),

    ("They speak different languages", "Ils parlent différentes langues"),

    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),

    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),

    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),

    ("The dog barks loudly", "Le chien aboie bruyamment"),

    ("He sings beautifully", "Il chante magnifiquement"),

    ("They swim in the pool", "Ils nagent dans la piscine"),

    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),

    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),

    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),

    ("He paints landscapes", "Il peint des paysages"),

    ("They laugh at the joke", "Ils rient de la blague"),

    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),

    ("She runs in the park", "Elle court dans le parc"),

    ("We travel by train", "Nous voyageons en train"),

    ("He writes a letter", "Il écrit une lettre"),

    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),

    ("The baby cries", "Le bébé pleure"),

    ("She studies hard for exams", "Elle étudie dur pour les examens"),

    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),

    ("He fixes the car", "Il répare la voiture"),

    ("They drink coffee in the morning", "Ils boivent du café le matin"),

    ("The sun sets in the evening", "Le soleil se couche le soir"),

    ("She dances at the party", "Elle danse à la fête"),

    ("We play music at the concert", "Nous jouons de la musique au concert"),

    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),

    ("They study French grammar", "Ils étudient la grammaire française"),

    ("The rain falls gently", "La pluie tombe doucement"),

    ("She sings a song", "Elle chante une chanson"),

    ("We watch a movie together", "Nous regardons un film ensemble"),

    ("He sleeps deeply", "Il dort profondément"),

    ("They travel to Paris", "Ils voyagent à Paris"),

    ("The children play in the park", "Les enfants jouent dans le parc"),

    ("She walks along the beach", "Elle se promène le long de la plage"),

    ("We talk on the phone", "Nous parlons au téléphone"),

    ("He waits for the bus", "Il attend le bus"),

    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),

    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),

    ("She dreams of flying", "Elle rêve de voler"),

    ("We work in the office", "Nous travaillons au bureau"),

    ("He studies history", "Il étudie l'histoire"),

    ("They listen to the radio", "Ils écoutent la radio"),

    ("The wind blows gently", "Le vent souffle doucement"),

    ("She swims in the ocean", "Elle nage dans l'océan"),

    ("We dance at the wedding", "Nous dansons au mariage"),

    ("He climbs the mountain", "Il gravit la montagne"),

    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),

    ("The cat meows loudly", "Le chat miaule bruyamment"),

    ("She paints a picture", "Elle peint un tableau"),

    ("We build a sandcastle", "Nous construisons un château de sable"),

    ("He sings in the choir", "Il chante dans le chœur")
]


In [134]:
class Vocab:
  def __init__(self):
    self.word2idx = {"": 0, "": 1, "": 2}
    self.idx2word = {0: "", 1: "", 2: ""}
    self.word_count = {}
    self.n_words = 3

  def add_sentence(self, sentence):
    for word in sentence.split(' '):
      self.add_word(word)

  def add_word(self, word):
    if word not in self.word2idx:
      self.word2idx[word] = self.n_words
      self.idx2word[self.n_words] = word
      self.word_count[word] = 1
      self.n_words += 1
    else:
      self.word_count[word] += 1



In [135]:
def tokenize(sentences, vocab):
  maxlen = max(len(sentence.split(' ')) for sentence in sentences) + 2
  tokenizedsentences = []
  for sentence in sentences:
    tokens = [vocab.word2idx[""]] + [vocab.word2idx[word] for word in sentence.split(' ')] + [vocab.word2idx[""]]
    paddedtokens = tokens + [vocab.word2idx[""]] * (maxlen - len(tokens))
    tokenizedsentences.append(paddedtokens)
  return torch.tensor(tokenizedsentences, dtype = torch.long)



In [136]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [137]:
def pairToTensors(pair):
  inputtens = torch.tensor(pair[0], dtype=torch.long)
  targettens = torch.tensor(pair[1], dtype=torch.long)
  return (inputtens, targettens)

In [138]:
class EFDataset(Dataset):
  def __init__(self, pairs):
    self.engvocab = Vocab()
    self.fravocab = Vocab()
    self.pairs = []

    for eng, fra in pairs:
      self.engvocab.add_sentence(eng)
      self.fravocab.add_sentence(fra)
      self.pairs.append((eng, fra))

    self.eng_sentences = [pair[0] for pair in self.pairs]
    self.fra_sentences = [pair[1] for pair in self.pairs]

    self.eng_tokens = tokenize(self.eng_sentences, self.engvocab)
    self.fra_tokens = tokenize(self.fra_sentences, self.fravocab)

    self.eng_embedding = torch.nn.Embedding(self.engvocab.n_words, 100)
    self.fra_embedding = torch.nn.Embedding(self.fravocab.n_words, 100)

  def __len__(self):
    return len(self.pairs)

  def __getitem__(self, idx):
    eng_tokens = self.eng_tokens[idx]
    fra_tokens = self.fra_tokens[idx]

    eng_emb = self.eng_embedding(eng_tokens)
    fra_emb = self.fra_embedding(fra_tokens)

    return eng_tokens, fra_tokens, eng_emb, fra_emb






In [139]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def forward(self, input, hidden):
    embedded = self.embedding(input).view(1, 1, -1)
    output, hidden = self.gru(embedded, hidden)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [140]:
class Decoder(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden):
    embedded = self.embedding(input).view(1, 1, -1)
    output, hidden = self.gru(embedded, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden

  def init_hidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [141]:
dataset = EFDataset(engtofra)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
val_dataset = EFDataset(engtofra)

val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [142]:
encoder = Encoder(input_size=dataset.engvocab.n_words, hidden_size=256).to(device)
decoder = Decoder(hidden_size=256, output_size=dataset.fravocab.n_words).to(device)

hiddensize = 256
lr = .01

encoderoptim = optim.SGD(encoder.parameters(), lr=lr)
decoderoptim = optim.SGD(decoder.parameters(), lr=lr)

In [143]:
def training(input_tensor, target_tensor, encoder, decoder, encoderoptim, decoderoptim, criterion, maxleng = 12):
  encoder_hidden = encoder.initHidden()

  encoderoptim.zero_grad()
  decoderoptim.zero_grad()

  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)

  encoder_outputs = torch.zeros(maxleng, encoder.hidden_size, device=device)

  loss = 0

  for ei in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

  decoder_input = torch.tensor([dataset.fravocab.word2idx['']], device=device)
  decoder_hidden = encoder_hidden

  for di in range(target_length):
    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
    topv, topi = decoder_output.topk(1)
    decoder_input = topi.squeeze().detach()

    loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
    if decoder_input == torch.tensor([dataset.fravocab.word2idx['']], device=device):
      break

  loss.backward()

  encoderoptim.step()
  decoderoptim.step()

  return loss.item() / target_length


In [144]:
criterion = nn.NLLLoss()

epochs = 50


In [145]:
for epoch in range(epochs):
  totalloss = 0
  for eng_tokens, fra_tokens, _, _ in dataloader:
    input_tensor = eng_tokens.squeeze().to(device)
    target_tensor = fra_tokens.squeeze().to(device)
    loss = training(input_tensor, target_tensor, encoder, decoder, encoderoptim, decoderoptim, criterion)
    totalloss += loss

  if epoch % 10 == 0:
    print(f"Epoch {epoch} loss {totalloss / len(dataloader)}")

Epoch 0 loss 0.13057824524082098
Epoch 10 loss 9.648953618068323e-05
Epoch 20 loss 4.761031740306575e-05
Epoch 30 loss 3.140164415380635e-05
Epoch 40 loss 2.3356303690856826e-05


In [146]:
##Part 2

class AttentionDecoder(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(AttentionDecoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout = .1
    self.maxleng = 12

    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.gru = nn.GRU(self.hidden_size, self.hidden_size)
    self.out = nn.Linear(self.hidden_size, self.output_size)
    self.attn = nn.Linear(self.hidden_size, self.maxleng)
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout)

  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    attn_weights = F.softmax(self.attn(hidden[0]), dim=1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)

    output = F.relu(output)
    output, hidden = self.gru(output, hidden)

    output = F.log_softmax(self.out(output[0]), dim=1)
    return output, hidden, attn_weights

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [147]:
encoder = Encoder(input_size=dataset.engvocab.n_words, hidden_size=256).to(device)
decoder = AttentionDecoder(hidden_size=256, output_size=dataset.fravocab.n_words).to(device)

hiddensize = 256
lr = .01

encoderoptim = optim.SGD(encoder.parameters(), lr=lr)
decoderoptim = optim.SGD(decoder.parameters(), lr=lr)

In [148]:
def attentiontraining(input_tensor, target_tensor, encoder, decoder, encoderoptim, decoderoptim, criterion, encoder_ouputs, maxleng = 12):
  encoder_hidden = encoder.initHidden()

  encoderoptim.zero_grad()
  decoderoptim.zero_grad()

  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)

  encoder_outputs = torch.zeros(maxleng, encoder.hidden_size, device=device)

  loss = 0

  for ei in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

  decoder_input = torch.tensor([dataset.fravocab.word2idx['']], device=device)
  decoder_hidden = encoder_hidden

  for di in range(target_length):
    decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
    topv, topi = decoder_output.topk(1)
    decoder_input = topi.squeeze().detach()

    loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
    if decoder_input == torch.tensor([dataset.fravocab.word2idx['']], device=device):
      break

  loss.backward()

  encoderoptim.step()
  decoderoptim.step()

  return loss.item() / target_length


In [149]:
criterion = nn.NLLLoss()
epochs = 50

In [150]:
for epoch in range(epochs):
  totalloss = 0
  for eng_tokens, fra_tokens, _, _ in dataloader:
    input_tensor = eng_tokens.squeeze().to(device)
    target_tensor = fra_tokens.squeeze().to(device)
    input_length = input_tensor.size(0)
    encoder_hidden = encoder.initHidden()
    encoder_outputs = torch.zeros(12, encoder.hidden_size, device=device)
    for ei in range(input_length):
         encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
         encoder_outputs[ei] = encoder_output[0, 0]

    loss = attentiontraining(input_tensor, target_tensor, encoder, decoder, encoderoptim, decoderoptim, criterion, encoder_outputs)
    totalloss += loss

  if epoch % 10 == 0:
    print(f"Epoch {epoch} loss {totalloss / len(dataloader)}")

Epoch 0 loss 0.1153460418257902
Epoch 10 loss 9.158525599849576e-05
Epoch 20 loss 4.640159677107157e-05
Epoch 30 loss 3.218800194054038e-05
Epoch 40 loss 2.269744257811828e-05


In [151]:
##Part 3

fratoeng = [
    ("J'ai froid", "I am cold"),

    ("Tu es fatigué", "You are tired"),

    ("Il a faim", "He is hungry"),

    ("Elle est heureuse", "She is happy"),

    ("Nous sommes amis", "We are friends"),

    ("Ils sont étudiants", "They are students"),

    ("Le chat dort", "The cat is sleeping"),

    ("Le soleil brille", "The sun is shining"),

    ("Nous aimons la musique", "We love music"),

    ("Elle parle français couramment", "She speaks French fluently"),

    ("Il aime lire des livres", "He enjoys reading books"),

    ("Ils jouent au football chaque week-end", "They play soccer every weekend"),

    ("Le film commence à 19 heures", "The movie starts at 7 PM"),

    ("Elle porte une robe rouge", "She wears a red dress"),

    ("Nous cuisinons le dîner ensemble", "We cook dinner together"),

    ("Il conduit une voiture bleue", "He drives a blue car"),

    ("Ils visitent souvent des musées", "They visit museums often"),

    ("Le restaurant sert une délicieuse cuisine", "The restaurant serves delicious food"),

    ("Elle étudie les mathématiques à l'université", "She studies mathematics at university"),

    ("Nous regardons des films le vendredi", "We watch movies on Fridays"),

    ("Il écoute de la musique en faisant du jogging", "He listens to music while jogging"),

    ("Ils voyagent autour du monde", "They travel around the world"),

    ("Le livre est sur la table", "The book is on the table"),

    ("Elle danse avec grâce", "She dances gracefully"),

    ("Nous célébrons les anniversaires avec un gâteau", "We celebrate birthdays with cake"),

    ("Il travaille dur tous les jours", "He works hard every day"),

    ("Ils parlent différentes langues", "They speak different languages"),

    ("Les fleurs fleurissent au printemps", "The flowers bloom in spring"),

    ("Elle écrit de la poésie pendant son temps libre", "She writes poetry in her free time"),

    ("Nous apprenons quelque chose de nouveau chaque jour", "We learn something new every day"),

    ("Le chien aboie bruyamment", "The dog barks loudly"),

    ("Il chante magnifiquement", "He sings beautifully"),

    ("Ils nagent dans la piscine", "They swim in the pool"),

    ("Les oiseaux gazouillent le matin", "The birds chirp in the morning"),

    ("Elle enseigne l'anglais à l'école", "She teaches English at school"),

    ("Nous prenons le petit déjeuner ensemble", "We eat breakfast together"),

    ("Il peint des paysages", "He paints landscapes"),

    ("Ils rient de la blague", "They laugh at the joke"),

    ("L'horloge tic-tac bruyamment", "The clock ticks loudly"),

    ("Elle court dans le parc", "She runs in the park"),

    ("Nous voyageons en train", "We travel by train"),

    ("Il écrit une lettre", "He writes a letter"),

    ("Ils lisent des livres à la bibliothèque", "They read books at the library"),

    ("Le bébé pleure", "The baby cries"),

    ("Elle étudie dur pour les examens", "She studies hard for exams"),

    ("Nous plantons des fleurs dans le jardin", "We plant flowers in the garden"),

    ("Il répare la voiture", "He fixes the car"),

    ("Ils boivent du café le matin", "They drink coffee in the morning"),

    ("Le soleil se couche le soir", "The sun sets in the evening"),

    ("Elle danse à la fête", "She dances at the party"),

    ("Nous jouons de la musique au concert", "We play music at the concert"),

    ("Il cuisine le dîner pour sa famille", "He cooks dinner for his family"),

    ("Ils étudient la grammaire française", "They study French grammar"),

    ("La pluie tombe doucement", "The rain falls gently"),

    ("Elle chante une chanson", "She sings a song"),

    ("Nous regardons un film ensemble", "We watch a movie together"),

    ("Il dort profondément", "He sleeps deeply"),

    ("Ils voyagent à Paris", "They travel to Paris"),

    ("Les enfants jouent dans le parc", "The children play in the park"),

    ("Elle se promène le long de la plage", "She walks along the beach"),

    ("Nous parlons au téléphone", "We talk on the phone"),

    ("Il attend le bus", "He waits for the bus"),

    ("Ils visitent la tour Eiffel", "They visit the Eiffel Tower"),

    ("Les étoiles scintillent la nuit", "The stars twinkle at night"),

    ("Elle rêve de voler", "She dreams of flying"),

    ("Nous travaillons au bureau", "We work in the office"),

    ("Il étudie l'histoire", "He studies history"),

    ("Ils écoutent la radio", "They listen to the radio"),

    ("Le vent souffle doucement", "The wind blows gently"),

    ("Elle nage dans l'océan", "She swims in the ocean"),

    ("Nous dansons au mariage", "We dance at the wedding"),

    ("Il gravit la montagne", "He climbs the mountain"),

    ("Ils font de la randonnée dans la forêt", "They hike in the forest"),

    ("Le chat miaule bruyamment", "The cat meows loudly"),

    ("Elle peint un tableau", "She paints a picture"),

    ("Nous construisons un château de sable", "We build a sandcastle"),

    ("Il chante dans le chœur", "He sings in the choir")

]


In [152]:
dataset = EFDataset(fratoeng)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [153]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def forward(self, input, hidden):
    embedded = self.embedding(input).view(1, 1, -1)
    output, hidden = self.gru(embedded, hidden)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [154]:

class AttentionDecoder(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(AttentionDecoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout = .1
    self.maxleng = 12

    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.gru = nn.GRU(self.hidden_size, self.hidden_size)
    self.out = nn.Linear(self.hidden_size, self.output_size)
    self.attn = nn.Linear(self.hidden_size, self.maxleng)
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout)

  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    attn_weights = F.softmax(self.attn(hidden[0]), dim=1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)

    output = F.relu(output)
    output, hidden = self.gru(output, hidden)

    output = F.log_softmax(self.out(output[0]), dim=1)
    return output, hidden, attn_weights

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [155]:
encoder = Encoder(input_size=dataset.engvocab.n_words, hidden_size=256).to(device)
decoder = AttentionDecoder(hidden_size=256, output_size=dataset.fravocab.n_words).to(device)

hiddensize = 256
lr = .01

encoderoptim = optim.SGD(encoder.parameters(), lr=lr)
decoderoptim = optim.SGD(decoder.parameters(), lr=lr)

In [156]:
def attentiontraining(input_tensor, target_tensor, encoder, decoder, encoderoptim, decoderoptim, criterion, encoder_ouputs, maxleng = 12):
  encoder_hidden = encoder.initHidden()

  encoderoptim.zero_grad()
  decoderoptim.zero_grad()

  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)

  encoder_outputs = torch.zeros(maxleng, encoder.hidden_size, device=device)

  loss = 0

  for ei in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

  decoder_input = torch.tensor([dataset.fravocab.word2idx['']], device=device)
  decoder_hidden = encoder_hidden

  for di in range(target_length):
    decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
    topv, topi = decoder_output.topk(1)
    decoder_input = topi.squeeze().detach()

    loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
    if decoder_input == torch.tensor([dataset.fravocab.word2idx['']], device=device):
      break

  loss.backward()

  encoderoptim.step()
  decoderoptim.step()

  return loss.item() / target_length


In [157]:
criterion = nn.NLLLoss()
epochs = 50


In [158]:
for epoch in range(epochs):
  totalloss = 0
  for eng_tokens, fra_tokens, _, _ in dataloader:
    input_tensor = eng_tokens.squeeze().to(device)
    target_tensor = fra_tokens.squeeze().to(device)
    input_length = input_tensor.size(0)
    encoder_hidden = encoder.initHidden()
    encoder_outputs = torch.zeros(12, encoder.hidden_size, device=device)
    for ei in range(input_length):
         encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
         encoder_outputs[ei] = encoder_output[0, 0]

    loss = attentiontraining(input_tensor, target_tensor, encoder, decoder, encoderoptim, decoderoptim, criterion, encoder_outputs)
    totalloss += loss

  if epoch % 10 == 0:
    print(f"Epoch {epoch} loss {totalloss / len(dataloader)}")

Epoch 0 loss 0.11321698739979566
Epoch 10 loss 0.00012281693863880002
Epoch 20 loss 6.121367314046931e-05
Epoch 30 loss 3.755520942720382e-05
Epoch 40 loss 3.0927477439080625e-05
